In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import string
from tqdm import tqdm

%matplotlib inline
from IPython.core.pylabtools import figsize

In [2]:
tf.__version__

'1.7.0'

In [3]:
tf.logging.set_verbosity("WARN")

In [4]:
textfile = "/media/joe/HD/data/gutenberg/zarathustra.txt"
rawtext = open(textfile, "r").read()
rawtext = rawtext.lower()

In [5]:
characters = list(set(rawtext))
len(characters)

61

In [6]:
char_map = {characters[i]:i for i in range(len(characters))}

In [7]:
len(rawtext)

658812

In [8]:
tf.reset_default_graph()

In [9]:
def train_input_function(text=rawtext[:600000], seq_len=50, cmap=char_map, batch_size=256, N=100):
    """
    """
    def _gen():
        for _ in range(batch_size*N):
            start = np.random.randint(0, len(text)-seq_len-1)
            sampletext = np.array([char_map[t] for t in text[start:start+seq_len]])
            label = char_map[text[start+seq_len]]
            yield sampletext, label
    ds = tf.data.Dataset.from_generator(_gen, output_types=(tf.int64, tf.int64),
                                       output_shapes=((seq_len), ()))
    ds = ds.batch(batch_size)
    return ds.make_one_shot_iterator().get_next()

In [10]:
def test_input_function(text=rawtext[600000:], seq_len=50, cmap=char_map, batch_size=256, N=10):
    """
    """
    def _gen():
        
        for i in range(batch_size*N):
            start = (seq_len*i)%(len(text)-seq_len-1)
            sampletext = np.array([char_map[t] for t in text[start:start+seq_len]])
            label = char_map[text[start+seq_len]]
            yield sampletext, label
    ds = tf.data.Dataset.from_generator(_gen, output_types=(tf.int64, tf.int64),
                                       output_shapes=((seq_len), ()))
    ds = ds.batch(batch_size)
    return ds.make_one_shot_iterator().get_next()

In [11]:
def model_fn(features, labels, mode, params):
    """
    """
    is_training = mode == tf.estimator.ModeKeys.TRAIN
    N = len(params["char_map"])
    x_oh = tf.one_hot(features, N)
    
    # params["rnn_cells"] = [128,128]
    cells = [tf.nn.rnn_cell.BasicRNNCell(n) for n in params["rnn_cells"]]
    mcell = tf.nn.rnn_cell.MultiRNNCell(cells)
    outputs, state = tf.nn.dynamic_rnn(mcell, x_oh, dtype=tf.float32)
    
    dense = tf.layers.dense(outputs[:,-1,:], params["hidden"], activation=tf.nn.relu)
    dropout = tf.layers.dropout(dense, params["dropout"], training=is_training)
    logits = tf.layers.dense(dropout, N)
    
    probs = tf.nn.softmax(logits)
    predicted_class = tf.argmax(logits, 1)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {"probs":probs, "class":predicted_class}
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
    
    y_oh = tf.one_hot(labels, N)
    loss = tf.losses.softmax_cross_entropy(y_oh, logits)
    
    # compute eval metrics
    accuracy = tf.metrics.accuracy(labels=labels, predictions=predicted_class)
    metrics={"accuracy":accuracy}
    tf.summary.scalar("accuracy", accuracy[1])
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics
        )   
    
    optimizer = tf.train.AdamOptimizer(params["learn_rate"])
    #optimizer = tf.train.MomentumOptimizer(params["learn_rate"], 0.9)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [12]:
def RNNEstimator(char_map, rnn_cells=[128,128], hidden=128, dropout=0.5, learn_rate=1e-3, model_dir="logs"):
    """
    
    """
    params = {"rnn_cells":rnn_cells, "hidden":hidden, "dropout":dropout,
             "learn_rate":learn_rate, "char_map":char_map}
    return tf.estimator.Estimator(model_fn=model_fn, params=params, model_dir=model_dir)

In [13]:
model = RNNEstimator(char_map, rnn_cells=[256,256], hidden=512)

In [14]:
#model.train(train_input_function)

In [15]:
#model.train(train_input_function)

In [16]:
#model.evaluate(test_input_function)

In [17]:
train_spec = tf.estimator.TrainSpec(input_fn=train_input_function, max_steps=10000)
eval_spec = tf.estimator.EvalSpec(input_fn=test_input_function)

In [ ]:
tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

In [19]:
model.evaluate(test_input_function)

{'accuracy': 0.30781251, 'global_step': 10000, 'loss': 2.519465}

In [20]:
def generate_text(mod, test_text="it was a dark and stormy night", N=200, seq_len=25):
    test_text = test_text.lower()
    if len(test_text) < seq_len:
        n = seq_len-len(test_text)
        test_text = " "*n + test_text
    for _ in tqdm(range(N)):
        inpt = np.array([char_map[i] for i in test_text[-seq_len:]]).reshape(1,-1)
        input_fn = tf.estimator.inputs.numpy_input_fn(inpt, num_epochs=1, shuffle=False)
        p = list(mod.predict(input_fn, predict_keys=["probs"]))[0]["probs"]
        test_text += np.random.choice(characters, p=p)
    return test_text

In [21]:
print(generate_text(model))

100%|██████████| 200/200 [03:03<00:00,  1.09it/s]

it was a dark and stormy night mw aut 
aml. the cha sraald
. bdeo geleeg
iy hid thing ange and atesh cedi 7ith wins thugtiu thid if sut iurt$af sed tt it as ema )ircentye
-.lamethto-d
maty thour te-dken, curir go  ly therheshc ill
